In [53]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [54]:
def date_change(date):
    nowdate = datetime.strptime(date, "%Y-%m-%d")

    nextYear = nowdate.replace(year=nowdate.year + 1)
    endDate = (nextYear - timedelta(days=1)).strftime("%Y%m%d")
    startDate = nowdate.strftime("%Y%m%d")

    return startDate, endDate

In [55]:
date_change("2024-05-22")

('20240522', '20250521')

In [56]:
def parse_month(year, month, stnid, stnnm, m_temp, m_gtemp):

    try:

        # 지점 이름, 지점 아이디

        return {
            "년도": year,
            "월": month,
            "지점 번호": stnid,
            "지점명": stnnm,
            "월평균 기온": m_temp,
            "월평균 지면온도": m_gtemp,
        }

    except AttributeError as e:
        return {
            "년도": None,
            "월": None,
            "지점 번호": None,
            "지점명": None,
            "월평균 기온": None,
            "월평균 지면온도": None,
        }

In [57]:
def mean_cul(item, result_m):

    STNID = item.find("stnId").get_text()
    STNNM = item.find("stnNm").get_text()
    try:
        temp = float(item.find("avgTa").get_text())
        ground_temp = float(item.find("avgTs").get_text())
    except ValueError:
        temp = 0
        ground_temp = 0
        if "error" in result_m:
            result_m["error"] = result_m["error"] + 1
        else:
            result_m["error"] = 1

    result_m["stnid"] = STNID
    result_m["stnnm"] = STNNM

    if "t_total" in result_m:
        result_m["t_total"] = result_m["t_total"] + temp
        result_m["gt_total"] = result_m["gt_total"] + ground_temp
        result_m["cnt"] = result_m["cnt"] + 1
    else:
        result_m["t_total"] = temp
        result_m["gt_total"] = ground_temp
        result_m["cnt"] = 1

    return result_m

In [58]:
def init(result_m):
    result_m = {}
    return result_m


In [59]:
def data_calling(day, id, month_result):
    startDt, endDt = date_change(day)

    url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    params = {
        "serviceKey": "JblClLk+uLRS6hB9k3LbHV8cJYUxFo3XL4pNRgX5yJOdGf8E8wlFkrB8so5AunCF9QnI/tIdf/vkDbpqcWwzKg==",
        "pageNo": 1,
        "numOfRows": "365",
        "dataType": "XML",
        "dataCd": "ASOS",
        "dateCd": "DAY",
        "startDt": startDt,
        "endDt": endDt,
        "stnIds": id,
    }

    response = requests.get(url, params=params)

    # pprint.pprint(response.text, depth=4)

    soup = BeautifulSoup(response.content, features="xml")
    # pprint.pprint(soup.text)
    items = soup.find_all("item")

    if items is None:
        print("nothing in items")

    month = "01"

    result_m = {}
    # result_m = init(result_m)

    for item in items:
        TM = item.find("tm").get_text()

        months = TM.split("-")

        if month == months[1]:
            result_m = mean_cul(item, result_m)

        else:
            if "error" in result_m:
                count = result_m["cnt"] - result_m["error"]
            else:
                count = result_m["cnt"]

            m_temp = result_m["t_total"] / count
            m_gtemp = result_m["gt_total"] / count
            month_result.append(
                parse_month(
                    months[0],
                    month,
                    result_m["stnid"],
                    result_m["stnnm"],
                    m_temp,
                    m_gtemp,
                )
            )
            month = months[1]
            result_m = {}
            # result_m = init(result_m)
            result_m = mean_cul(item, result_m)

    if "error" in result_m:
        count = result_m["cnt"] - result_m["error"]
    else:
        count = result_m["cnt"]

    m_temp = result_m["t_total"] / count
    m_gtemp = result_m["gt_total"] / count

    month_result.append(
        parse_month(
            months[0], month, result_m["stnid"], result_m["stnnm"], m_temp, m_gtemp
        )
    )

    return month_result

In [60]:
import pandas as pd

data = pd.read_csv('./observation_point_info.csv',encoding="CP949")
OPI_df = pd.DataFrame(data)

In [61]:
observation_point_num_list = [90,93,95,98,99,100,101,102,104,105,106,108,112,114,115,119,121,127,129,130,131,133,135,136,137,138,140,143,146,152,155,156,159,162,165,168,169,170,172,174,177,184,185,188,189,192,201,202,203,211,212,216,217,221,226,232,235,236,238,239,243,244,245,247,248,251,252,253,254,255,257,258,259,260,261,262,263,264,266,268,271,272,273,276,277,278,279,281,283,284,285,288,289,294,295]

In [62]:
opi2_df = OPI_df[OPI_df['위도']<37]

In [73]:
len(opi2_df)

110

In [63]:
final_opi_df = opi2_df[opi2_df['위도']>36]

In [64]:
len(final_opi_df)

34

In [65]:
opi_point_list = list(final_opi_df['지점'])

In [66]:
len(opi_point_list)

34

In [69]:
opi_final_list = []
for point in opi_point_list:
    if point in observation_point_num_list:
        opi_final_list.append(point)

In [74]:
len(opi_final_list)

32

In [70]:
month_result = []
date_list = [
    "2014-01-01",
    "2015-01-01",
    "2016-01-01",
    "2017-01-01",
    "2018-01-01",
    "2019-01-01",
    "2020-01-01",
    "2021-01-01",
    "2022-01-01",
    "2023-01-01",
]
# id_list = ["127"]
id_list = opi_final_list

In [71]:
for date in date_list:
    for id in id_list:
        month_result = data_calling(date, id, month_result)


df = pd.DataFrame(month_result)

KeyError: 'cnt'

---
에러가 발생했는데 수정이 힘들거 같아서 다시 만듦

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'bs4'

In [3]:
def date_change(date):

    nowdate = datetime.strptime(date, "%Y-%m-%d")
    nextYear = nowdate.replace(year=nowdate.year + 1)
    
    endDate = (nextYear - timedelta(days=1)).strftime("%Y%m%d")
    startDate = nowdate.strftime("%Y%m%d")

    return startDate, endDate

In [4]:
# 함수 테스트
date_change("2024-05-22")

('20240522', '20250521')

In [5]:
def params_settig(date,opi):

    startDt , endDt = date_change(date)
    result = {
        'serviceKey' : 'fF/Ekuf8awXAsiiMYAivKxzIu1Hyl76PXGGqz88Dvs7sMLoKbSaWzAxF2//kihhIurY9PYr5kxC6lEvw8mDC4Q==',
        'pageNo' : '1',
        'numOfRows' : '365',
        'dataType' : 'XML',
        'dataCd' : 'ASOS',
        'dateCd' : 'DAY',
        'startDt' : startDt,
        'endDt' : endDt,
        'stnIds' : opi
    }

    return result

In [6]:
# 함수 테스트
params_settig("2024-05-24","123")

{'serviceKey': 'fF/Ekuf8awXAsiiMYAivKxzIu1Hyl76PXGGqz88Dvs7sMLoKbSaWzAxF2//kihhIurY9PYr5kxC6lEvw8mDC4Q==',
 'pageNo': '1',
 'numOfRows': '365',
 'dataType': 'XML',
 'dataCd': 'ASOS',
 'dateCd': 'DAY',
 'startDt': '20240524',
 'endDt': '20250523',
 'stnIds': '123'}

In [7]:
def get_api(url, params):
    response = requests.get(url, params=params)
    contents = response.content
    decode_xml= contents.decode('UTF-8')
    return decode_xml.replace('/','').replace('>','').split('<')

In [8]:
def get_data(list,char):
    result = []
    for i in list:
        char_len = len(char)
        if char in i:
            result.append(i[char_len:])
    return [item for item in result if item and item.strip()]

In [9]:
def data_filter(data,dict):
    result = {}
    for a,b in dict.items():
        result = {**result,**{a:get_data(data,b)}}
    return result

In [3]:
# 유효 관측 지점 찾기
data = pd.read_csv('./observation_point_info.csv',encoding="CP949")
OPI_df = pd.DataFrame(data)
# 우리가 가지고 있는 데이터의 관측 지점 숫자
observation_point_num_list = [90,93,95,98,99,100,101,102,104,105,106,108,112,114,115,119,121,127,129,130,131,133,135,136,137,138,140,143,146,152,155,156,159,162,165,168,169,170,172,174,177,184,185,188,189,192,201,202,203,211,212,216,217,221,226,232,235,236,238,239,243,244,245,247,248,251,252,253,254,255,257,258,259,260,261,262,263,264,266,268,271,272,273,276,277,278,279,281,283,284,285,288,289,294,295]

#유효 위도 36~37
opi2_df = OPI_df[OPI_df['위도']<37]
final_opi_df = opi2_df[opi2_df['위도']>36]
opi_point_list = list(final_opi_df['지점'])
opi_final_list = []
for point in opi_point_list:
    if point in observation_point_num_list:
        opi_final_list.append(point)
opi_final_list = sorted(set(opi_final_list))

In [4]:
opi_final_list = sorted(set(opi_final_list))

In [5]:
opi_final_list

[127,
 129,
 130,
 131,
 133,
 135,
 136,
 137,
 138,
 140,
 177,
 226,
 232,
 235,
 236,
 238,
 239,
 271,
 272,
 273,
 276,
 277,
 278,
 279]

In [6]:
len(opi_final_list)

24

In [12]:
# 177 빼기
opi_final_list = sorted(set(opi_final_list))

In [13]:
remove_set = {133,177,239}

opi_valid_list = [i for i in opi_final_list if i not in remove_set]
opi_valid_list

[129,
 130,
 131,
 135,
 136,
 137,
 138,
 140,
 271,
 272,
 273,
 276,
 277,
 278,
 279,
 226,
 232,
 235,
 236,
 238,
 127]

In [16]:
# 유효한 숫자들인지 확인
opn_txt_list = '''90속초 	212홍천
93북춘 	216태백 
95철원 	217정선군 
98동두천 	221제천 
99파주 	226보은 
100대관령 	232천안 
101춘천 	235보령
102백령도 	236부여 
104북강릉 	238금산 
105강릉 	239세종 
106동해 	243부안 
108서울 	244임실 
112인천 	245정읍 
114원주 	247남원 
115울릉도 	248장수 
119수원	251고창군
121영월 	252영광군
127충주 	253김해시
129서산	254순창군
130울진	255북창원
131청주	257양산시 
133대전	258보성군 
135추풍령	259강진군 
136안동 	260장흥 
137상주	261해남 
138포항 	262고흥 
140군산 	263의령군 
143대구 	264함양군 
146전주 	266광양시 
152울산 	268진도군 
155창원 	271봉화 
156광주 	272영주 
159부산 	273문경 
162통영 	276청송군 
165목포 	277영덕 
168여수 	278의성 
169흑산도 	279구미 
170완도 	281영천 
172고창 	283경주시 
174순천 	284거창 
177홍성 	285합천 
184제주 	288밀양 
185고산 	289산청 
188성산 	294거제 
189서귀포 	295남해 
192진주 
201강화 
202양평 
203이천 
211인제 '''

for observation_point_num in opi_final_list:
    check = str(observation_point_num)
    if check in opn_txt_list:
        continue
    else:
        print(observation_point_num)

In [26]:
opi_valid_list.remove(271)

In [27]:
len(opi_valid_list)

20

In [28]:
opi_valid_list1 = opi_valid_list[:10]
opi_valid_list2 = opi_valid_list[10:]

In [29]:
date_list = ["2014-01-01","2015-01-01","2016-01-01","2017-01-01","2018-01-01","2019-01-01","2020-01-01","2021-01-01","2022-01-01","2023-01-01"]
opn_list = opi_valid_list
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
dict = {
    '지점번호' : "stnId",
    '지점명' : 'stnNm',
    '시간' : 'tm',
    '평균 기온' : 'avgTa',
    '평균 지면온도' : 'avgTs'
}

# for date in date_list:
date = "2014-01-01"
for opn in opn_list:
    params = params_settig(date=date, opi=opn)
    data = get_api(url,params)
    filt_data = data_filter(data,dict)
    df = pd.DataFrame(filt_data)
    print(df)

    지점번호 지점명          시간 평균 기온 평균 지면온도
0    129  서산  2014-01-01   3.0     1.8
1    129  서산  2014-01-02   0.7     1.8
2    129  서산  2014-01-03   2.3     2.1
3    129  서산  2014-01-04  -1.1     1.4
4    129  서산  2014-01-05  -2.0     0.7
..   ...  ..         ...   ...     ...
360  129  서산  2014-12-27  -1.8     0.8
361  129  서산  2014-12-28  -0.7     0.4
362  129  서산  2014-12-29   1.3     1.3
363  129  서산  2014-12-30   1.4     1.4
364  129  서산  2014-12-31  -0.5     1.2

[365 rows x 5 columns]
